In [1]:
import numpy as np 
import pandas as pd 
from simple_salesforce import Salesforce
from datetime import date  

In [2]:
## Initializations

source_org_url = 'https://unitedwaynashville.my.salesforce.com' 
source_org_session_id = ''
destination_org_url = 'https://marthaobryan.my.salesforce.com'
destination_org_session_id = ''

source_org = Salesforce(instance_url=source_org_url, session_id=source_org_session_id)
destination_org = Salesforce(instance_url=destination_org_url, session_id=destination_org_session_id)

household_id = '0014N00001ncL4bQAE'
record_origin = 'SHFS'

In [3]:
#HH Account Lambda Functinos
def name_lambda(row):
    new_name = row['Name'][:row['Name'].find("(")-1]
    new_name += ' Household'
    return new_name

def emergency_contact_lambda(row):
    if(str(row['BillingCity'])=='' or pd.isna(row['BillingCity'])):
        return ''
    else:
        return str(row['BillingStreet']) + ", " + str(row['BillingCity']) + ", " + str(row['BillingState']) + " " + str(int(row['BillingPostalCode']))

def living_situation_lambda(row):
    if(row['Client_Current_Living_Situation__c']=='Substance abuse treatment facility or detox center' or row['Client_Current_Living_Situation__c']=='Interim housing' or row['Client_Current_Living_Situation__c']=='Data not collected'):
        return 'Other'
    else:
        return(row['Client_Current_Living_Situation__c'])

In [4]:
# Find Record Type on Dest Org
record_type = destination_org.query("SELECT Id FROM RecordType WHERE DeveloperName='HH_Account' AND SobjectType='Account'")
data = source_org.query("Select Id, Name, Website, Phone, Type, Email__c,  Emergency_Contact_Name__c, Email_Emergency_Contact__c, Phone_Emergency_Contact__c, ShippingStreet, ShippingCity, ShippingState, ShippingPostalCode, Living_Situation_Current__c  from Account WHERE Id='{}'".format(household_id))

# Transform Data Frame for Insert
df = pd.DataFrame(data['records']).drop(['attributes'],axis=1)
df.insert(1,'Record_Origin__c',record_origin)
df = df.rename(columns={'Id':'External_Id__c', 'Email_Emergency_Contact__c': 'Emergency_Contact_Email__c', 'Living_Situation_Current__c': 'Client_Current_Living_Situation__c', 'Phone_Emergency_Contact__c': 'Emergency_Contact_Phone__c', 'ShippingStreet':'BillingStreet', 'ShippingCity':'BillingCity', 'ShippingState':'BillingState', 'ShippingPostalCode':'BillingPostalCode'})
df.insert(3,'RecordTypeId',record_type['records'][0]['Id'])
df['Name'] = df.apply(lambda row: name_lambda(row), axis=1)
df['Client_Current_Living_Situation__c'] = df.apply(lambda row: living_situation_lambda(row), axis=1)
df.insert(12,'Emergency_Contact_Address__c','')
df['Emergency_Contact_Address__c'] = df.apply(lambda row: emergency_contact_lambda(row), axis=1)

In [5]:
#insert records
records_to_insert = df.to_dict('records')
household = []

#Save household id (Don't actually need the loop now) 

for record in records_to_insert:
    result = destination_org.Account.create(record)
    household.append(result)
    
new_household_id = household[0]['id']

In [11]:
# Start of CONTACTS

def ethnicity_lambda(row):
    if(row['Ethnicity__c']=='Non-Hispanic/Non-Latino/Non-Latinx/Non-Latine'):
        return 'Non-Hispanic/Non-Latino'
    elif(row['Ethnicity__c']=='Hispanic/Latino/Latina/Latinx/Latine' or row['Ethnicity__c']=='Hispanic/Latin(a)(o)(x)'):
        return 'Hispanic/Latino'
    elif(row['Ethnicity__c']=='Hispanic/Latino/Latinx/Latine'):
        return 'Hispanic/Latino'
    elif(row['Ethnicity__c']=='Non-Hispanic/Latin(a)(o)(x)'):
        return 'Non-Hispanic/Non-Latino'
    else:
        return row['Ethnicity__c']

def race_lambda(row):
    if(row['Race__c']=='Black, African American, or African'):
        return 'Black/African American'
    else:
        return row['Race__c']

def secondary_race_lambda(row):
    if(row['Secondary_Race__c']=='Black, African American, or African'):
        return 'Black/African American'
    else:
        return row['Secondary_Race__c']

def rel_to_head_lambda(row):
    if(row['Relationship_to_Head_of_Household__c']=='Self (head of household)'):
        return 'Self'
    else:
        return row['Relationship_to_Head_of_Household__c']


In [12]:
record_type = destination_org.query("SELECT Id FROM RecordType WHERE DeveloperName='Client' AND SobjectType='Contact'")
data = source_org.query("Select Id,Birthdate,Email,Ethnicity__c,FirstName,Gender__c,LastName,MailingCity,MailingPostalCode,MailingState,MailingStreet,Phone,Race__c,Relationship_to_Head_of_Household__c,Salutation,Secondary_Race__c,Veteran_Status__c FROM Contact WHERE AccountId='{}'".format(household_id))

df = pd.DataFrame(data['records']).drop(['attributes'],axis=1)

account_id = destination_org.query("SELECT Id from Account WHERE External_Id__c='{}'".format(household_id))

df.insert(1,'Record_Origin__c',record_origin)
df.insert(2,'AccountId',new_household_id)
df = df.rename(columns={'Id':'External_Id__c'})
df.insert(3,'RecordTypeId',record_type['records'][0]['Id'])
#df.insert(3,'RecordTypeId','0123t000000S2eoAAC') MOBC RecordType which will not show up in query
df.insert(4,'Client_Doesn_t_Know__c',"true")
df.insert(5,'Current_member_of_household__c',"Yes")
df['Ethnicity__c'] = df.apply(lambda row: ethnicity_lambda(row), axis=1)
df['Race__c'] = df.apply(lambda row: race_lambda(row), axis=1)
df['Secondary_Race__c'] = df.apply(lambda row: secondary_race_lambda(row), axis=1)
df['Relationship_to_Head_of_Household__c'] = df.apply(lambda row: rel_to_head_lambda(row), axis=1)


In [13]:
# Insert contacts
new_contacts = []

records_to_insert = df.to_dict('records')

for record in records_to_insert:
    result = destination_org.Contact.create(record)
    new_contacts.append(result)
    
new_contact_ids = []

for i in new_contacts:
    new_contact_ids.append(i['id'])
                           
df['Id'] = new_contact_ids
new_contacts = df # Save contact ID in destination org to a new DF named new_contacts



In [14]:
# BEGIN PE TFC
# PE TFC Lambda Functions

def salesforce_roi_lambda(row):
    if(row['Salesforce_ROI_Uploaded__c']==0):
        return 'N/A'
    elif(row['Salesforce_ROI_Uploaded__c']==1):
        return 'Uploaded'

def verbal_consent_lambda(row):
    if(row['Verbal_Consent_Only__c']==0):
        return 'N/A'
    elif(row['Verbal_Consent_Only__c']==1):
        return 'Uploaded'

def mnps_roi_lambda(row):
    if(row['MNPS_ROI__c']==0):
        return 'N/A'
    elif(row['MNPS_ROI__c']==1):
        return 'Uploaded'

def hmis_roi_lambda(row):
    if(row['HMIS_ROI_Uploaded_del__c']==0):
        return 'N/A'
    elif(row['HMIS_ROI_Uploaded_del__c']==1):
        return 'Uploaded'

def name_lambda(row):
    month_loc = row['Open_Date__c'].find('/')
    month = row['Open_Date__c'][:month_loc]
    day_loc = month_loc + 1 + row['Open_Date__c'][month_loc+1:].find('/')
    day = row['Open_Date__c'][month_loc+1:day_loc]
    if (len(month) == 1):
        month = '0' + month
    if (len(day) == 1):
        day = '0' + day

    year = row['Open_Date__c'][day_loc+1:]
    # open_date = date(int(year), int(month), int(day)).isoformat()
    # new_name = row['Household/Organization Name'] + open_date
    return row['Name'] + ": " + str(year) + "-" + str(month) + "-" + str(day)

In [15]:
record_type = destination_org.query("SELECT Id FROM RecordType WHERE DeveloperName='Family_Collective_Enrollment' AND SobjectType='pmdm__ProgramEngagement__c'")
data = source_org.query("SELECT Household_Organization__r.Id, Household_Organization__r.Salesforce_ROI_Uploaded__c, Household_Organization__r.Verbal_Consent_Only__c, Household_Organization__r.MNPS_ROI__c, 	Household_Organization__r.HMIS_ROI_Uploaded_del__c, Household_Organization__r.Name, Open_Date__c, Living_Situation_at_Entry__c, Closure_Date__c FROM Enrollment_History__c WHERE Household_Organization__r.Id='{}'".format(household_id))
program = destination_org.query("Select Id from pmdm__Program__c WHERE Name='The Family Collective'")

In [ ]:

data = source_org.query("SELECT Household_Organization__r.Id, Household_Organization__r.Salesforce_ROI_Uploaded__c, Household_Organization__r.Verbal_Consent_Only__c, Household_Organization__r.MNPS_ROI__c, 	Household_Organization__r.HMIS_ROI_Uploaded_del__c, Household_Organization__r.Name, Open_Date__c, Living_Situation_at_Entry__c, Closure_Date__c FROM Enrollment_History__c WHERE Household_Organization__r.Id='{}'".format(household_id))

household_fields = data['records'][0]['Household_Organization__r']
household_fields.pop('attributes')

household_df = pd.DataFrame(household_fields,index=[0])


In [ ]:

enrollment_data = source_org.query("SELECT Open_Date__c, Living_Situation_at_Entry__c, Closure_Date__c FROM Enrollment_History__c WHERE Household_Organization__r.Id='{}'".format(household_id))
enrollment_data
enrollment_fields = enrollment_data['records'][0]
enrollment_fields.pop('attributes')
enrollment_df = pd.DataFrame(enrollment_fields,index=[0])
df = pd.concat([household_df, enrollment_df],axis=1, ignore_index=False)

In [18]:
df.insert(1,'Record_Origin__c',record_origin)
df.insert(3,'RecordTypeId',record_type['records'][0]['Id'])
df.insert(4,'pmdm__Program__c',program['records'][0]['Id'])
df.insert(6,'pmdm__Account__c',new_household_id)
df.insert(8,'pmdm__Stage__c','Active')

df['Salesforce_ROI_Uploaded__c'] = df.apply(lambda row: salesforce_roi_lambda(row), axis=1)
# df['Verbal Consent Given (No ROI)'] = df.apply(lambda row: verbal_consent_lambda(row), axis=1)
df['MNPS_ROI__c'] = df.apply(lambda row: mnps_roi_lambda(row), axis=1)
df['HMIS_ROI_Uploaded_del__c'] = df.apply(lambda row: hmis_roi_lambda(row), axis=1)
df['Name'] = df.apply(lambda row: name_lambda(row), axis=1)

In [19]:
for index, row in new_contacts.iterrows():
    if (row['Relationship_to_Head_of_Household__c']=="Self"):
        df.insert(9,'pmdm__Contact__c',row['Id'])
        
df = df.rename(columns={"Living_Situation_at_Entry__c":"Living_Situation_Prior_to_Entry__c", "Open_Date__c":"pmdm__StartDate__c","Salesforce_ROI_Uploaded__c":"HMIS_Salesforce__c","Verbal_Consent_Only__c":"Verbal_Consent_Given_Salesforce_ROI__c","HMIS_ROI_Uploaded_del__c":"HMIS_ROI__c"})
df = df.drop('Id',axis=1)


In [20]:
new_pe_TFC = [] 
records_to_insert = df.to_dict('records')

for record in records_to_insert:
    result = destination_org.pmdm__programengagement__c.create(record)
    new_pe_TFC.append(result)
    
new_pe_TFC_ids = [] #only one should be created right now

for i in new_pe_TFC:
    new_pe_TFC_ids.append(i['id'])

In [21]:
# Start of PE Participant 

data = source_org.query("SELECT Id,AccountId, School_Name__c,School_Type__c,Current_childcare_child_education_status__c, enrolled_in_early_learning_program__c,Grade_Level_Child_at_Entry__c,Bankruptcy__c,FirstName,LastName,Birthdate,Relationship_to_head_of_household__c FROM Contact WHERE Account.Id='{}'".format(household_id))
acct_data = source_org.query("Select Name, Id, Open_Date_From_Most_Recent_Enrollment__c from Account WHERE Account.Id='{}'".format(household_id))
record_type = destination_org.query("SELECT Id FROM RecordType WHERE DeveloperName='Family_Collective_Participant_Enrollment' AND SobjectType='pmdm__ProgramEngagement__c'")



In [22]:
df = pd.DataFrame(data['records']).drop(['attributes'],axis=1)
acct_df = pd.DataFrame(acct_data['records']).drop(['attributes'],axis=1) 


In [23]:
acct_df = acct_df.rename(columns={'Id':'AccountId'})
df = pd.merge(df, acct_df, on='AccountId')
df = df.rename(columns={'Id':'External_Id__c'})

df['Relationship_to_Head_of_Household__c'] = df.apply(lambda row: rel_to_head_lambda(row), axis=1)
df.insert(1,'Record_Origin__c',record_origin)
df.insert(3,'RecordTypeId',record_type['records'][0]['Id'])
df.insert(4,'pmdm__Program__c',program['records'][0]['Id'])
df.insert(6,'pmdm__Account__c',new_household_id)
df.insert(8,'pmdm__Stage__c','Active')
df.insert(9,'Family_Collective_Enrollment__c',new_pe_TFC_ids[0])
df.insert(10,'Client_Doesn_t_Know__c',True)


              
# insert account id and contact ids

In [24]:
new_contacts_ids = new_contacts[['External_Id__c','Id']]
new_contacts_ids = new_contacts_ids.rename(columns={'Id': 'pmdm__Contact__c'})

df = pd.merge(new_contacts_ids, df, on='External_Id__c')
del df['External_Id__c']
del df['AccountId']
df.head()
df = df.rename(columns={'Enrolled_in_Early_Learning_Program__c': 'Enrolled_in_TFC_Childcare__c', 
                        'Grade_Level_Child_at_Entry__c': 'Grade_Level_at_Entry__c',
                        'Open_Date_From_Most_Recent_Enrollment__c' : 'pmdm__StartDate__c',
                        'FirstName': 'First_Name__c',
                        'LastName': 'Last_Name__c',
                        'Birthdate': 'Birthdate__c'})
del df['Current_childcare_child_education_status__c']



In [25]:
new_pe_participants = [] 
records_to_insert = df.to_dict('records')

for record in records_to_insert:
    result = destination_org.pmdm__programengagement__c.create(record)
    new_pe_participants.append(result)
    


In [26]:
# Start of Case Notes

def add_creator_to_case_note(row):
    return row['Case_Note__c'] + " -" + row['CreatedByName']

def add_case_note_name(row):
    return row['Date__c'] + " Case Note"

data = source_org.query("SELECT Case_Note__c,Date__c,Household__c,Id,CreatedById FROM Case_Note__c WHERE Household__c='{}' AND Client_Interaction__c=null".format(household_id))

df = pd.DataFrame(data['records']).drop(['attributes'],axis=1)

df.insert(1,'Record_Origin__c',record_origin)
del df['Household__c']
df.insert(2,'Household__c',new_pe_TFC_ids[0])

df = df.rename(columns={'Id':'External_Id__c'})

users = source_org.query("SELECT Id,Name from User")
users_df = pd.DataFrame(users['records']).drop(['attributes'],axis=1)
users_df = users_df.rename(columns={'Id': 'CreatedById', 'Name': 'CreatedByName'})
df = pd.merge(df, users_df, on='CreatedById')

#df.insert(6,'New_Case_Note__c','')
df['Name'] = df.apply(lambda row: add_case_note_name(row), axis=1)
#df['New_Case_Note__c'] = df.apply(lambda row: add_creator_to_case_note(row),axis=1)
#del df['Case_Note__c']
del df['CreatedByName']
del df['CreatedById']
#df = df.rename(columns={'New_Case_Note__c': 'Case_Note__c'})

In [27]:
records_to_insert = df.to_dict('records')

for record in records_to_insert:
    destination_org.case_note__c.create(record)

In [28]:
# Start of Sessions

def add_session_note_name(row):
    return row['Session_Date__c'] + " Case Note"

def session_format_lambda(row):
    if(row['Session_Format__c']=='Video Conference/Zoom'):
        return 'Video Conference'
    else:
        return row['Session_Format__c']

#query for sessions related to household_id
data = source_org.query("SELECT Care_Team_Member__c,Case_Note_for_session__c,Household__c,Id,Name,RecordTypeId,Services_Rendered__c,Session_Date__c,Session_Format__c,Session_Length__c FROM Session__c WHERE Household__c='{}'".format(household_id))
df = pd.DataFrame(data['records']).drop(['attributes'],axis=1)

#add record origin, replace household with PE TFC Id for destination org
df.insert(1,'Record_Origin__c',record_origin)
del df['Household__c']
df.insert(2,'pmdm__programengagement__c',new_pe_TFC_ids[0])

#get ids for care team member user records in source org
users = source_org.query("SELECT Id,Name from User")
users = pd.DataFrame(users['records']).drop(['attributes'],axis=1)

#get ids for care team member contacts in destination org
care_team_members = destination_org.query("SELECT Name,Id FROM Contact WHERE RecordType.Name='Care Team Member'")
care_team_members = pd.DataFrame(care_team_members['records']).drop(['attributes'],axis=1)

users = users.rename(columns={'Id':'SourceOrgUserId'})
care_team_members = care_team_members.rename(columns={'Id':'DestOrgContactId'})

care_team_members = pd.merge(care_team_members, users, on='Name')

#get id for recordType in dest org
record_type = destination_org.query("SELECT Id FROM RecordType WHERE DeveloperName='Session' AND SobjectType='pmdm__ServiceDelivery__c'")
del df['RecordTypeId']

df.insert(1,'RecordTypeId',record_type['records'][0]['Id'])
df = df.rename(columns={'Id':'External_Id__c', 'Care_Team_Member__c':'SourceOrgUserId'})
df = pd.merge(df, care_team_members, on='SourceOrgUserId')
df = df.rename(columns={'DestOrgContactId': 'Primary_Care_Team_Member__c','Case_Note_for_session__c':'Session_Notes__c','Services_Rendered__c':'Services_Received_select_all_that_apply__c','Session_Length__c':'pmdm__quantity__c'})
del df['SourceOrgUserId']
del df['Name_x']
del df['Name_y']
df.insert(2,'Name','')
df['Name'] = df.apply(lambda row: add_session_note_name(row), axis=1)
df['Session_Format__c'] = df.apply(lambda row: session_format_lambda(row), axis=1)
df = df.replace({np.nan: None})

In [29]:
records_to_insert = df.to_dict('records')

for record in records_to_insert:
    destination_org.pmdm__servicedelivery__c.create(record)

In [32]:
# begin client goals

data = source_org.query("SELECT Date_Goal_Achieved__c,Date_Goal_Set__c,Detailed_Goal_Description__c,Goal_Status__c,Household_Account__c,Id,Outcome__c,Pillar__c,Specific_Goal_Action_Step__c FROM Client_Goal__c WHERE Household_Account__c='{}'".format(household_id))

df = pd.DataFrame(data['records']).drop(['attributes'],axis=1)

df.insert(1,'Record_Origin__c',record_origin)
df = df.rename(columns={'Id':'External_Id__c','Detailed_Goal_Description__c':'Detailed_description_of_goal_optional__c', 'Pillar__c':'Domain__c'})
df.insert(2,'Household__c',new_pe_TFC_ids[0])
del df['Household_Account__c']
records_to_insert = df.to_dict('records')

for record in records_to_insert:
    destination_org.client_goal__c.create(record)